# Possible Endpoints
- audio-features
- artists
- albums
- tracks

In [4]:
#System
import os
import json
import zipfile
#API
import requests
#Data Processing
from datetime import date, datetime
import pandas as pd
import numpy as np
import math
import time
#SQL
import psycopg2
import psycopg2.extras as extras 
#Security
from getpass import getpass

In [2]:
class SpotifyTerminal:
    def __init__(self):
        AUTH_URL = 'https://accounts.spotify.com/api/token'
        
        # POST
        auth_response = requests.post(AUTH_URL, {
            'grant_type': 'client_credentials',
            'client_id': os.getenv('SPOTIPY_CLIENT_ID'),
            'client_secret': os.getenv('SPOTIPY_CLIENT_SECRET'),
        })
        
        # convert the response to JSON
        auth_response_data = auth_response.json()
        
        # save the access token
        access_token = auth_response_data['access_token']
        
        self.headers = {
            'Authorization': 'Bearer {token}'.format(token=access_token)
        }
        self.init_timestamp = datetime.now().strftime("%Y%m%d_%H_%M_%S")
        self.BASE_URL = 'https://api.spotify.com/v1/'

    def __repr__(self):
        return 'initialized at: {}'.format(self.init_timestamp)

    def call_api_id(self, id, endpoint):
        # turn a list of IDs into a string for the API
        if type(id) == list:
            id = '?ids='+'%2C'.join(id)
        else:
            id = '/' + id
            
        # actual GET request with proper header
        r = requests.get(self.BASE_URL + endpoint + id, headers=self.headers)
        # r.status_code
        return r


    def call_api_id_additional(self, id, endpoint, additional, offset=None):
        # turn a list of IDs into a string for the API
        id = '/' + id
            
        # actual GET request with proper header
        if offset == None:
            r = requests.get(self.BASE_URL + endpoint + id + '/' + additional + '?include_groups=album&limit=50', headers=self.headers)
        else:
            r = requests.get(self.BASE_URL + endpoint + id + '/' + additional + '?include_groups=album&limit=50&offset=' + offset, headers=self.headers)
        # r.status_code
        return r

    def call_api_search(self, track_name, artist_name):
        track_name_formatted = track_name.replace(' ', '%20')
        artist_name_formatted = artist_name.replace(' ', '%20')
        endpoint = 'search?q=track%3A{}%20artist%3A{}&type=track&limit=1'.format(track_name_formatted, artist_name_formatted)
        print(self.BASE_URL + endpoint)
        # actual GET request with proper header
        r = requests.get(self.BASE_URL + endpoint, headers=self.headers)
        # r.status_code
        return r


# opens json files provided by spotify data retreival
def get_json_from_file(json_file):
    f = open(json_file, encoding="utf8")
    data = json.load(f)
    f.close()
    return data

# SQL

In [1]:
class DatabaseTerminal:
    def __init__(self):
        self.password = getpass()
        
    def open(self):
        self.conn = psycopg2.connect(database = "melodybox_db",
                                user = "postgres",
                                password = self.password,
                                host = "localhost",
                                port = "5432")
        # self.conn = psycopg2.connect(database = "melodybox_db",
        #                         user = "badmin",
        #                         password = self.password,
        #                         host = "melodybox.c920sk60g6ei.us-east-2.rds.amazonaws.com",
        #                         port = "5432")
        # self.conn = psycopg2.connect(database = "music_hub",
        #                         user = "postgres",
        #                         password = self.password,
        #                         host = "localhost",
        #                         port = "5432")

    def close(self):
        self.conn.close()
  
    def query(self, query):
        # Returns a dataframe of the query result
        cursor = self.conn.cursor() 
        df_query_result = 0
        try:
            cursor.execute(query)
            query_result = cursor.fetchall()
            col_names = [desc[0] for desc in cursor.description]
            df_query_result = pd.DataFrame(data=query_result, columns=col_names)
        except (Exception, psycopg2.DatabaseError) as error: 
            print("Error: %s" % error) 
            cursor.close() 
        cursor.close()
        return df_query_result
        
    def execute_values(self, df, table): 
        # Prep Data
        tuples = [tuple(x) for x in df.to_numpy()] 
        cols = ','.join(list(df.columns)) 
        # SQL query to execute 
        query = "INSERT INTO %s(%s) VALUES %%s" % (table, cols) 
        cursor = self.conn.cursor() 
        # Execute
        try: 
            extras.execute_values(cursor, query, tuples) 
            self.conn.commit() 
        except (Exception, psycopg2.DatabaseError) as error: 
            print("Error: %s" % error) 
            self.conn.rollback() 
            cursor.close() 
            return 1
        print("the dataframe is inserted") 
        cursor.close()

        

In [14]:
staging_path = 'processing/data_staging/'
os.listdir(staging_path)[0]

'extended_history_2024-01-19.zip'

In [18]:
class DataProcessor:
    def __init__(self):
        self.staging_path = 'processing/data_staging/'
        
    def get_db_table_ids(self):
        print('input db password')
        self.db = DatabaseTerminal()
        self.db.open()
        self.db_stream_list = list(self.db.query('SELECT id FROM main.streams'))
        self.db_track_list = list(self.db.query('SELECT id FROM main.tracks'))
        self.db_artist_list = list(self.db.query('SELECT id FROM main.artists'))
        self.db_album_list = list(self.db.query('SELECT id FROM main.albums'))
        self.db_genre_map = self.db.query('SELECT raw_genre, main_genre FROM builder.genres')
        self.db.close()

    def print_db_stats(self):
        print('total streams: {}'.format(len(self.db_stream_list)))
        print('total tracks: {}'.format(len(self.db_track_list)))
        print('total artists: {}'.format(len(self.db_artist_list)))
        print('total albums: {}'.format(len(self.db_album_list)))

    def check_files_in_staging(self):
        files_in_staging = os.listdir(self.staging_path)
        num_files_in_staging = len(files_in_staging)
        if num_files_in_staging > 1:
            print('too many files in staging')
            self.staging_file = None
            return
        elif num_files_in_staging == 0:
            print('no files in staging')
            self.staging_file = None
            return
        else:
            self.staging_file = self.staging_path + files_in_staging[0]
            print('staging_file: {}'.format(self.staging_file))
    
    def normalize_raw_extended_streams(self):
        # Open and append streaming audio hisotry 
        if self.staging_file == None:
            print('normalizer could not find staging file')
            return
        streaming_data_raw = []
        with zipfile.ZipFile(self.staging_file, "r") as myzip:
            for filename in myzip.namelist():
                if 'Streaming_History_Audio' in filename:
                    with myzip.open(filename) as f: 
                        data = f.read()
                        d = json.loads(data)
                        streaming_data_raw.extend(d)
        df_streams = pd.DataFrame(streaming_data_raw)
        
        # Create a DataFrame for tracks and streams
        
        df_streams['all_uri'] = np.where(df_streams["spotify_track_uri"].isnull(), df_streams["spotify_episode_uri"], df_streams["spotify_track_uri"] )
        df_streams['track_name'] = np.where(df_streams["master_metadata_track_name"].isnull(), df_streams["episode_name"], df_streams["master_metadata_track_name"] )
        df_streams['type'] = np.where(df_streams["master_metadata_track_name"].isnull(), 'episode', 'track' )
        df_streams = df_streams[df_streams['all_uri'].notnull()]
        df_streams['track_id'] = df_streams.apply(lambda row: row.all_uri[-22:], axis=1)
        stream_column_rename = {'master_metadata_album_artist_name':'artist_name', 
                                'master_metadata_album_album_name':'album_name',
                                'ms_played':'duration_ms',
                                'ts':'played_at'
                                }
        
        df_streams.rename(columns=stream_column_rename,inplace=True)
        df_streams['data_source'] = 'extended_history'
        self.raw_streams = df_streams[[
            'track_id',
            'track_name',
            'artist_name',
            'album_name',
            'type',
            'duration_ms',
            'played_at',
            'reason_start',
            'reason_end',
            'shuffle',
            'skipped',
            'username',
            'data_source'
                   ]]
        
        print('Number of streams from extended history: {}'.format(len(self.raw_streams)))

    def move_staging_to_archive(self):
        

In [19]:
proc = DataProcessor()
proc.check_files_in_staging()
proc.normalize_raw_extended_streams()
proc.raw_streams.head()

staging_file: processing/data_staging/extended_history_2024-01-19.zip


In [ ]:
proc = DataProcessor()
proc.get_db_table_ids()
proc.print_db_stats()

In [ ]:
#abandoned for now
class StreamHistory:
    def __init__(self, zip_file_name):
        self.zip_file_name = zip_file_name
        with zipfile.ZipFile(self.zip_file_name, "r") as myzip:
            for filename in myzip.namelist():
                if 'Streaming_History_Audio' in filename:
                    with myzip.open(filename) as f: 
                        data = f.read()
                        d = json.loads(data)
                        streaming_data_raw.extend(d)
                elif 'StreamingHistory_music' in filename:
        df_streams = pd.DataFrame(streaming_data_raw)

# Get Streaming History

### EXTENDED - STREAMS

In [13]:
# Location
zip_name = 'DATA_DUMP/extended_history_2024-01-19.zip'

# Open and append streaming audio hisotry 
streaming_data_raw = []
with zipfile.ZipFile(zip_name, "r") as myzip:
    for filename in myzip.namelist():
        if 'Streaming_History_Audio' in filename:
            with myzip.open(filename) as f: 
                data = f.read()
                d = json.loads(data)
                streaming_data_raw.extend(d)
df_streams = pd.DataFrame(streaming_data_raw)

# Create a DataFrame for tracks and streams

df_streams['all_uri'] = np.where(df_streams["spotify_track_uri"].isnull(), df_streams["spotify_episode_uri"], df_streams["spotify_track_uri"] )
df_streams['track_name'] = np.where(df_streams["master_metadata_track_name"].isnull(), df_streams["episode_name"], df_streams["master_metadata_track_name"] )
df_streams['type'] = np.where(df_streams["master_metadata_track_name"].isnull(), 'episode', 'track' )
df_streams = df_streams[df_streams['all_uri'].notnull()]
df_streams['track_id'] = df_streams.apply(lambda row: row.all_uri[-22:], axis=1)
stream_column_rename = {'master_metadata_album_artist_name':'artist_name', 
                        'master_metadata_album_album_name':'album_name',
                        'ms_played':'duration_ms',
                        'ts':'played_at'
                        }

df_streams.rename(columns=stream_column_rename,inplace=True)
df_streams['data_source'] = 'extended_history'
df_streams = df_streams[[
    'track_id',
    'track_name',
    'artist_name',
    'album_name',
    'type',
    'duration_ms',
    'played_at',
    'reason_start',
    'reason_end',
    'shuffle',
    'skipped',
    'username',
    'data_source'
           ]]

print('Number of streams: {}'.format(len(df_streams)))

Number of streams: 80280


In [14]:
df_streams.head()

,track_id,track_name,artist_name,album_name,type,duration_ms,played_at,reason_start,reason_end,shuffle,skipped,username,data_source
0,7H6ev70Weq6DdpZyyTmUXk,Say My Name,Destiny's Child,The Writing's On The Wall,track,271256,2016-03-24T21:22:44Z,clickrow,trackdone,False,None,brian.cross741,extended_history
1,6Iblet1QGgY8TLPZ7GxFPZ,Summertime,DJ Jazzy Jeff & The Fresh Prince,Greatest Hits,track,271907,2016-03-24T21:39:22Z,trackdone,trackdone,False,None,brian.cross741,extended_history
2,503OTo2dSqe7qk76rgsbep,Still D.R.E.,Dr. Dre,2001,track,136464,2016-03-24T22:03:21Z,trackdone,unexpected-exit-while-paused,False,None,brian.cross741,extended_history
3,503OTo2dSqe7qk76rgsbep,Still D.R.E.,Dr. Dre,2001,track,134049,2016-03-24T22:05:37Z,appload,trackdone,True,None,brian.cross741,extended_history
4,6uQKuonTU8VKBz5SHZuQXD,This Is How We Do It,Montell Jordan,This Is How We Do It,track,146004,2016-03-25T12:38:33Z,trackdone,unexpected-exit-while-paused,True,None,brian.cross741,extended_history


### Original - Account Data

In [80]:
# Open Streaming History File
folder_path = 'DATA_DUMP/account_data_2024-01-19/'
file_name = 'StreamingHistory_music_0.json'

streaming_data_raw = get_json_from_file(folder_path +  file_name)
print('Number of tracks: {}'.format(len(streaming_data_raw)))

# Create a DataFrame for tracks and streams
df_streams = pd.DataFrame(streaming_data_raw)
df_streams['uniqueName'] = df_streams['artistName'] + df_streams['trackName']
df_tracks = df_streams[['uniqueName','artistName','trackName']].drop_duplicates().reset_index(drop=True)

Number of tracks: 7124


# Get Existing DB Data

In [4]:
db = DatabaseTerminal()
db.open()

df_sql_tracks = db.query('SELECT id FROM main.tracks')
df_sql_artists = db.query('SELECT id FROM main.artists')
df_sql_albums = db.query('SELECT id FROM main.albums')
df_sql_genres = db.query('SELECT raw_genre, main_genre FROM builder.genres')

db.close()

 ········


### Check Missing Tracks

In [15]:
print('total streams: {}'.format(len(df_streams)))
print('total tracks: {}'.format(df_streams[df_streams['type']=='track']['track_id'].nunique()))
print('total episodes: {}'.format(df_streams[df_streams['type']=='episode']['track_id'].nunique()))
print('total tracks existing: {}'.format(df_streams[df_streams['track_id'].isin(df_sql_tracks['id'])]['track_id'].nunique()))
print('total tracks missing: {}'.format(df_streams[~df_streams['track_id'].isin(df_sql_tracks['id'])]['track_id'].nunique()))

total streams: 80280
total tracks: 25744
total episodes: 2796
total tracks existing: 0
total tracks missing: 28540


In [16]:
missing_tracks = list(df_streams[~df_streams['track_id'].isin(df_sql_tracks['id'])]['track_id'].drop_duplicates())
print(len(missing_tracks))

28540


# Call API

### Call API Using Track ID

In [5]:
def parse_album_result(item):
    subset = dict((k, item[k]) for k in ('id', 'name', 'album_type', 'release_date', 'release_date_precision', 'popularity', 'total_tracks', 'label'))
    return subset

def parse_artist_result(item):
    # collect the total number of genres for each artist
    item['genre_count'] = len(item['genres'])

    # create fields for the top 5 genres
    if len(item['genres']) > 4:
        item['genre_5'] = item['genres'][4]
    else:
        item['genre_5'] = None
    
    if len(item['genres']) > 3:
        item['genre_4'] = item['genres'][3]
    else:
        item['genre_4'] = None
    
    if len(item['genres']) > 2:
        item['genre_3'] = item['genres'][2]
    else:
        item['genre_3'] = None
    
    if len(item['genres']) > 1:
        item['genre_2'] = item['genres'][1]
    else:
        item['genre_2'] = None
    
    if len(item['genres']) > 0:
        item['genre_1'] = item['genres'][0]
    else:
        item['genre_1'] = None

    item['followers'] = item['followers']['total']
    
    subset = dict((k, item[k]) for k in ('id','name','popularity','followers','genre_count','genre_1','genre_2','genre_3','genre_4','genre_5'))
    return subset

def parse_track_result(item):
    try:
        result = item['track']
    except:
        result = item
    subset = dict((k, result[k]) for k in ('id', 'name', 'duration_ms', 'popularity', 'disc_number', 'track_number', 'explicit', 'type'))
    subset['album_id'] = result['album']['id']
    subset['artist_id'] = result['artists'][0]['id']
    subset['artist_count'] = len(result['artists'])
    try:
        subset['context'] = item['context']['uri']
    except:
        pass
    try:
        subset['played_at'] = item['played_at']
    except:
        pass
    return subset


def get_new_table_data(search_list, table_type):
    # table_type can be ['artists','albums','tracks']
    #LIST FRACTION is the number of times the length of the list can be divided by either 50 or 20
    #ARTISTS and TRACKS have a call limit of 50 at a time, and ALBUMS is 20
    if table_type == 'artists' or table_type == 'tracks' or table_type == 'episodes':
        list_fraction = math.ceil(len(search_list) / 50)
    elif table_type == 'albums':
        list_fraction = math.ceil(len(search_list) / 20)
    
    #INITIALIZE
    sp = SpotifyTerminal()
    iteration = 0
    final = []

    #ITERATE
    for search_list_group in np.array_split(search_list, list_fraction):
        search_list_group = search_list_group.tolist()
        
        #CALL OPERATION
        result = sp.call_api_id(search_list_group, table_type)
        
        #EXCEPTION FOR ERRORS
        if result.status_code != 200:
            print('bad call on iteration: {}'.format(iteration))
            print('    status code: {}'.format(result.status_code))
            break
            
        #PARSE AND APPEND
        for i in result.json()[table_type]:
            if i is not None:
                if table_type == 'tracks':
                    result_clean = parse_track_result(i)
                    final.append(result_clean)
                elif table_type == 'artists':
                    result_clean = parse_artist_result(i)
                    final.append(result_clean)
                elif table_type == 'albums':
                    result_clean = parse_album_result(i)
                    final.append(result_clean)
            else:
                pass
        iteration += 1
    return final

In [18]:
new_track_data = get_new_table_data(list(missing_tracks), 'tracks')
df_tracks = pd.DataFrame(new_track_data)

In [19]:
len(new_track_data)

25744

### Expand to Artists and Albums

TEMPORARY
df_tracks = pd.read_csv('quicksave.csv')
df_tracks.drop('Unnamed: 0',axis=1,inplace=True)

In [20]:
print('Number of artists: {}'.format(df_tracks['artist_id'].nunique()))
print('Number of albums: {}'.format(df_tracks['album_id'].nunique()))
print('total artists existing: {}'.format(df_tracks[df_tracks['artist_id'].isin(df_sql_artists['id'])]['artist_id'].nunique()))
print('total artists missing: {}'.format(df_tracks[~df_tracks['artist_id'].isin(df_sql_artists['id'])]['artist_id'].nunique()))
print('total albums existing: {}'.format(df_tracks[df_tracks['album_id'].isin(df_sql_albums['id'])]['album_id'].nunique()))
print('total albums missing: {}'.format(df_tracks[~df_tracks['album_id'].isin(df_sql_albums['id'])]['album_id'].nunique()))

Number of artists: 5821
Number of albums: 12223
total artists existing: 0
total artists missing: 5821
total albums existing: 0
total albums missing: 12223


In [21]:
missing_artists = list(df_tracks[~df_tracks['artist_id'].isin(df_sql_artists['id'])]['artist_id'].drop_duplicates())
missing_albums = list(df_tracks[~df_tracks['album_id'].isin(df_sql_albums['id'])]['album_id'].drop_duplicates())

In [22]:
new_artist_data = get_new_table_data(list(missing_artists), 'artists')
df_artists = pd.DataFrame(new_artist_data)
df_artists = df_artists[['id','name','popularity','genre_count','genre_1','genre_2','genre_3','genre_4','genre_5','followers']]

In [25]:
new_album_data = get_new_table_data(list(missing_albums), 'albums')
df_albums = pd.DataFrame(new_album_data)
df_albums.rename(columns={'label':'album_label',inplace=True)
df_albums = df_albums[['id','name','album_type','release_date','release_date_precision','popularity','total_tracks','album_label']]

In [57]:
df_tracks = df_tracks[['id','name','disc_number','track_number','explicit','duration_ms','album_id','artist_id','artist_count','popularity']]

In [62]:
df_streams.rename(columns={'type':'track_type'},inplace=True)
my_list = ['track_id','track_name','artist_name','album_name','track_type','duration_ms','played_at','reason_start','reason_end','shuffle','skipped','username','data_source']
df_streams = df_streams[['track_id','track_name','artist_name','album_name','track_type','duration_ms','played_at','reason_start','reason_end','shuffle','skipped','username','data_source']]
df_streams = df_streams[df_streams['track_id'].isin(df_tracks['id'])]

In [69]:
db = DatabaseTerminal()
db.open()
db.execute_values(df_albums, 'main.albums')
db.execute_values(df_artists, 'main.artists')
db.execute_values(df_tracks, 'main.tracks')
db.execute_values(df_streams, 'main.streams')
db.close()

KeyboardInterrupt: Interrupted by user

In [73]:
db.close()

# Fill non-streamed tracks, albums, and artists

In [34]:
print('iteration: {} | {}'.format(1, str(datetime.now())))

iteration: 1 | 2024-02-16 11:07:28.635248


In [32]:
str(datetime.now())

'2024-02-16 11:06:52.841306'

In [43]:
def get_missing_id_list(sp, id, table_type, additional_info):
    iteration = 0
    list_amount = 50
    result_list = []
    
    while list_amount == 50:
        if iteration == 0:
            result = sp.call_api_id_additional(id, table_type, additional_info)
        else:
            result = sp.call_api_id_additional(id, table_type, additional_info, str(list_amount))
        if result.status_code != 200:
            if result.status_code == 429:
                print('bad call on iteration: {}'.format(id))
                print('    status code: {}'.format(result.status_code))
                print('    status header: {}'.format(result.headers))
                list_amount = 0
                print('    waiting for 5 minutes')
                time.sleep(300) #sleep for 5 minutes
                print('    ****RESUMING****')
            else:
                print('bad call on iteration: {}'.format(id))
                print('    status code: {}'.format(result.status_code))
                list_amount = 0
        else:
            for i in result.json()['items']:
                result_list.append(i['id'])
            list_amount = len(result.json()['items'])
            iteration += 1
    return result_list

In [ ]:
sp = SpotifyTerminal()
iteration = 0
list_amount = 50
result_list = []
all_albums = []

for artist_id in df_sql_artists['id']:
    # album_results = get_missing_id_list(artist_id, 'artists', 'albums')
    while list_amount == 50:
        if iteration == 0:
            result = sp.call_api_id_additional(id, table_type, additional_info)
        else:
            result = sp.call_api_id_additional(id, table_type, additional_info, str(list_amount))
        if result.status_code != 200:
            print('bad call on iteration: {}'.format(iteration))
            print('    status code: {}'.format(result.status_code))
            list_amount = 0
        elif result.status_code == 429:
            break
        else:
            for i in result.json()['items']:
                result_list.append(i['id'])
            list_amount = len(result.json()['items'])
            iteration += 1
    all_albums.extend(album_results)

In [44]:
print('# of existing artists: {}'.format(len(df_sql_artists)))
print('# of existing albums: {}'.format(len(df_sql_albums)))
print('# of existing tracks: {}'.format(len(df_sql_tracks)))

all_albums = []
sp = SpotifyTerminal()
iteration = 0
print('starting: {}'.format(str(datetime.now())))

for artist_id in df_sql_artists['id']:
    album_results = get_missing_id_list(sp, artist_id, 'artists', 'albums')
    all_albums.extend(album_results)
    iteration += 1
    if iteration % 200 == 0:
        print(iteration, str(datetime.now()))

# of existing artists: 5821
# of existing albums: 12223
# of existing tracks: 25744
starting: 2024-02-16 13:49:54.494960


KeyboardInterrupt: 

In [7]:
sp = SpotifyTerminal()
result = sp.call_api_id_additional('1W8TbFzNS15VwsempfY12H', 'artists', 'albums')

In [19]:
result.headers

{'content-type': 'application/json; charset=utf-8', 'cache-control': 'public, max-age=7200', 'x-robots-tag': 'noindex, nofollow', 'access-control-allow-origin': '*', 'access-control-allow-headers': 'Accept, App-Platform, Authorization, Content-Type, Origin, Retry-After, Spotify-App-Version, X-Cloud-Trace-Context, client-token, content-access-token', 'access-control-allow-methods': 'GET, POST, OPTIONS, PUT, DELETE, PATCH', 'access-control-allow-credentials': 'true', 'access-control-max-age': '604800', 'content-encoding': 'gzip', 'strict-transport-security': 'max-age=31536000', 'x-content-type-options': 'nosniff', 'date': 'Fri, 16 Feb 2024 15:48:47 GMT', 'server': 'envoy', 'Via': 'HTTP/2 edgeproxy, 1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000', 'Transfer-Encoding': 'chunked'}

In [25]:
requests.ConnectionError

requests.exceptions.ConnectionError

In [24]:
list(result.headers)

['content-type',
 'cache-control',
 'x-robots-tag',
 'access-control-allow-origin',
 'access-control-allow-headers',
 'access-control-allow-methods',
 'access-control-allow-credentials',
 'access-control-max-age',
 'content-encoding',
 'strict-transport-security',
 'x-content-type-options',
 'date',
 'server',
 'Via',
 'Alt-Svc',
 'Transfer-Encoding']

In [12]:
result

<Response [200]>

In [13]:
result.json()

{'href': 'https://api.spotify.com/v1/artists/1W8TbFzNS15VwsempfY12H/albums?include_groups=album&offset=0&limit=50',
 'items': [{'album_group': 'album',
   'album_type': 'album',
   'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1W8TbFzNS15VwsempfY12H'},
     'href': 'https://api.spotify.com/v1/artists/1W8TbFzNS15VwsempfY12H',
     'id': '1W8TbFzNS15VwsempfY12H',
     'name': 'Charles Mingus',
     'type': 'artist',
     'uri': 'spotify:artist:1W8TbFzNS15VwsempfY12H'}],
   'available_markets': ['AR',
    'AU',
    'AT',
    'BE',
    'BO',
    'BR',
    'BG',
    'CA',
    'CL',
    'CO',
    'CR',
    'CY',
    'CZ',
    'DK',
    'DO',
    'DE',
    'EC',
    'EE',
    'SV',
    'FI',
    'FR',
    'GR',
    'GT',
    'HN',
    'HK',
    'HU',
    'IS',
    'IE',
    'IT',
    'LV',
    'LT',
    'LU',
    'MY',
    'MT',
    'MX',
    'NL',
    'NZ',
    'NI',
    'NO',
    'PA',
    'PY',
    'PE',
    'PH',
    'PL',
    'PT',
    'SG',
    'SK',
    'ES

In [9]:
#INITIALIZE
table_type = 'artists'
additional_info = 'albums'
id = '1W8TbFzNS15VwsempfY12H'
test_results = get_missing_id_list(id, table_type, additional_info)

In [10]:
test_results

['2H4aJOMGjbsmcEuXkeAZij',
 '3uB6lXxaCmjy76Spz1Gv9A',
 '26RYXZiyXG698pvitJzel6',
 '28zHBSDrYtl1BqmPYHr1vR',
 '7rzqBkrED6EUBVOwaNpCKt',
 '3GsA6S0IkUGzE4xjv50256',
 '6wbXazLfhGxtO12C2KA0z3',
 '5OetjDwBwSH7C56zEnOd3x',
 '4iDGhH55WiYoz5Ho6G0fVG',
 '13UuZBeAf8hH5XfKJ1j0US',
 '4ISH6r7ClM5FKCJ7CDxC3X',
 '2poo6llLhnnItKte2EH7D9',
 '3ZlXHcNYckMxlM3Ha1C1z2',
 '1pLyXhcFmxcuak8KYK5ac1',
 '3Wgy8ip0ChdEe0EmvGykQr',
 '48hEJvkBze4dydhtIAjG1N',
 '7MmFqfWvSX7bUEQn62DLeM',
 '4mFHvcqplecJDsIT7dD1fe',
 '57ycb4xre9uujNxa01NKE7',
 '5jYSg1FIlxTBaDYXr0YVEh',
 '1LFY1dMtWrmQTLOvAedsU0',
 '7FfjuE9LfBDfRuizUG94rY',
 '1A8nmVkCixB6meKXWN9Okj',
 '6fqxA8YzUfpC6M406xSM6c',
 '446VZBiVSN8s59srcwBEtL',
 '7o9hJ8konXzGt6nKLy3MuX',
 '3QyDvbBVILY0chchx8aLtC',
 '254OYgcNQJIccVL79HIWxr',
 '2iMIXYXSs3XMdDkDSexI2d',
 '1XpvxdpyiuSTJTApwYjHgG',
 '03zH4bkGb2jSZAQIVW2px1',
 '6gRkcj3ntgkBWo7VosLJCa',
 '1QIzcLeA3XDZIbnWP5ALGA',
 '2E596WYlAsXZUZyv8q8IDg',
 '6Duqfxtoraff2mXG5oOZzj',
 '2sJJBMNuAcwEHteM3I65XY',
 '1LsS7KVPtRGDLsfyhZkSA2',
 

In [12]:
#INITIALIZE
table_type = 'albums'
additional_info = 'tracks'
id = '2H4aJOMGjbsmcEuXkeAZij'
test_results_2 = get_missing_id_list(id, table_type, additional_info)

In [13]:
test_results_2

['5NX3ITpPpleCqqH8w9waKi',
 '1fp7lFHvo4gJiymIl7vl44',
 '5i2gfAS7Tv1yVFcMly8Zs7',
 '4KtejgucKPOGGGJN0mz9xz',
 '237oN3hIWUDc3psqkvSEfI',
 '2fjOsaxvlTa7AsHgOlTcXx',
 '0uCw2ApQ5P5UI4qkMqCSvX',
 '11BnLY7aNfUqGtiWrkpfeJ',
 '5j1rVNd11W6op3IeLXJRzl',
 '6P4pThyhW8TbwcdUA5LK7x',
 '2D9JwqF2PrC6MSCeoN7yQr',
 '1kwOnvxjoYiKDwVRrx1S1W',
 '7xI5NiiuUZZIG8GVrRG3CG',
 '3UXH355xtCNG8aEUXRVrFH',
 '5u8c67yeZGhSVBr4ubCMMh',
 '6iWostsZ1kZc4DFC8FQ1gQ',
 '0Ou0Mxe99kdNTaFJSOsHTX',
 '0LLg2uZaEemTt79obVMfSV',
 '1bN6PosZ1jb88VwCIiYk3X',
 '0NlHyrLynRcLVSoPpNLa0L',
 '0d3JFFAEMNDcajRDYkbgVM',
 '3aSzTSESjQqMdqOe3LTr0w',
 '3W3S0szzKPwqFJMOIfM9aV',
 '0UUhelyg2GCU6jKjjFfPtK',
 '3kCBQcAvrL0DxN8h9WQEha',
 '1g2TzMRW8x5960YsGRqSnC',
 '1MBOG8fJp8ZkRcvoWda0jy',
 '0JNpZsPESZaaVveSEkcNSr',
 '00x74CXrEuc6fyL9oNRh0O',
 '4ECapBv6NuWJt8mc5hep0f',
 '0OdRgWYYRQtHnsqVoIeLGg',
 '6Dn6MuWye7hALlCxDrgv3a',
 '4citv7RrPPQO3DVbA604b8']

In [7]:
result_list

NameError: name 'result_list' is not defined

In [11]:
for i in df_sql_artists['id']:
    print(i)
    break

1Y8cdNmUJH7yBTd9yOvr5i


In [ ]:
df_sql_tracks
df_sql_artists
df_sql_albums
df_sql_genres

# Archive

In [ ]:
except (Exception, psycopg2.DatabaseError) as error: 
            print("Error: %s" % error) 
            self.conn.rollback() 
            self.cursor.close() 

In [207]:
len(new_track_data_clean)

51689

In [175]:
new_track_data_clean = []
iteration_tracker = []
iteration=0
for i in new_track_data:
    iteration_tracker.append(iteration)
    if i  is not None:
        new_track_data_clean.append(parse_track_result(i))
    iteration+=1
    # if iteration==2:
    #     break

In [176]:
max(iteration_tracker)

60533

In [177]:
new_track_data[2203:2205]

[{'album': {'album_type': 'album',
   'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0zokIKbH4c7crvZP9Xel1Y'},
     'href': 'https://api.spotify.com/v1/artists/0zokIKbH4c7crvZP9Xel1Y',
     'id': '0zokIKbH4c7crvZP9Xel1Y',
     'name': 'Richard Pryor',
     'type': 'artist',
     'uri': 'spotify:artist:0zokIKbH4c7crvZP9Xel1Y'}],
   'available_markets': [],
   'external_urls': {'spotify': 'https://open.spotify.com/album/2Zd4GXXqVsZiUA5vMJPGtq'},
   'href': 'https://api.spotify.com/v1/albums/2Zd4GXXqVsZiUA5vMJPGtq',
   'id': '2Zd4GXXqVsZiUA5vMJPGtq',
   'images': [{'height': 640,
     'url': 'https://i.scdn.co/image/ab67616d0000b273d616556599bf047d6ef91134',
     'width': 640},
    {'height': 300,
     'url': 'https://i.scdn.co/image/ab67616d00001e02d616556599bf047d6ef91134',
     'width': 300},
    {'height': 64,
     'url': 'https://i.scdn.co/image/ab67616d00004851d616556599bf047d6ef91134',
     'width': 64}],
   'name': 'The Anthology: 1968-1992',
   'releas

In [178]:
new_track_data_clean[2203:2205]

[{'id': '5VOS8q6SByertQVAKem8hx',
  'name': 'Exorcist',
  'duration_ms': 113893,
  'popularity': 0,
  'disc_number': 1,
  'track_number': 6,
  'explicit': True,
  'type': 'track',
  'album_id': '2Zd4GXXqVsZiUA5vMJPGtq',
  'artist_id': '0zokIKbH4c7crvZP9Xel1Y',
  'artist_count': 1},
 {'id': '7DOE1ATnFN3qcRMklFvqKh',
  'name': 'My Chain',
  'duration_ms': 238560,
  'popularity': 7,
  'disc_number': 1,
  'track_number': 2,
  'explicit': True,
  'type': 'track',
  'album_id': '77dd1df26ZbsefOUCVDk3n',
  'artist_id': '13y7CgLHjMVRMDqxdx0Xdo',
  'artist_count': 2}]

In [ ]:
sp = SpotifyTerminal()
tracks_list = []


for index, row in df_tracks.iterrows():
    r = sp.call_api_id(row['id'], 'tracks')
    if r.status_code != 200:
        print('bad call on iteration: {}'.format(index))
        print('    status code: {}'.format(r.status_code))
        break
    elif len(r.json()['tracks']['items']) > 0:
        track_parsed = sp.parse_track_result(r.json()['tracks']['items'][0])
        track_parsed['uniqueName'] = row['artistName'] + row['trackName']
        tracks_list.append(track_parsed)
    else:
        print('could not find {}'.format(row['artistName'] + row['trackName']))
        pass

# Call API Using Track Name and Artist Name

In [97]:
sp = SpotifyTerminal()
tracks_list = []
for index, row in df_tracks.iterrows():
    r = sp.call_api_search(row['trackName'],row['artistName'])
    if r.status_code != 200:
        print('bad call on iteration: {}'.format(index))
        print('    status code: {}'.format(r.status_code))
        break
    elif len(r.json()['tracks']['items']) > 0:
        track_parsed = sp.parse_track_result(r.json()['tracks']['items'][0])
        track_parsed['uniqueName'] = row['artistName'] + row['trackName']
        tracks_list.append(track_parsed)
    else:
        print('could not find {}'.format(row['artistName'] + row['trackName']))
        pass

https://api.spotify.com/v1/search?q=track%3AA%20Singularity%20-%20Re-Imagined%20by%20Carina%20Round%20artist%3APuscifer&type=track&limit=1
https://api.spotify.com/v1/search?q=track%3ASlip%20Slidin'%20Away%20artist%3APaul%20Simon&type=track&limit=1
https://api.spotify.com/v1/search?q=track%3ADragonaut%20artist%3ASleep&type=track&limit=1
https://api.spotify.com/v1/search?q=track%3AThe%20Mission%20-%20M%20Is%20For%20Milla%20Mix%20artist%3APuscifer&type=track&limit=1
https://api.spotify.com/v1/search?q=track%3AMomma%20Sed%20(Alive%20At%20Club%20Nokia)%20-%20Live%20artist%3APuscifer&type=track&limit=1
could not find PusciferMomma Sed (Alive At Club Nokia) - Live
https://api.spotify.com/v1/search?q=track%3AVagina%20Mine%20(Alive%20At%20Club%20Nokia)%20-%20Live%20artist%3APuscifer&type=track&limit=1
could not find PusciferVagina Mine (Alive At Club Nokia) - Live
https://api.spotify.com/v1/search?q=track%3APotions%20-%20Deliverance%20Mix%20artist%3APuscifer&type=track&limit=1
https://api.spoti

KeyboardInterrupt: 

In [95]:
tracks_list

[{'id': '79YU92loMkeK28GhVFAHsc',
  'name': 'A Singularity - Re-Imagined by Carina Round',
  'duration_ms': 290837,
  'popularity': 25,
  'disc_number': 1,
  'track_number': 9,
  'explicit': False,
  'type': 'track',
  'album_id': '2mgxGlqxz7sknQDovNf6bx',
  'artist_id': '2pAajGWerK3ghwToNWFENS',
  'artist_count': 2,
  'uniqueName': 'PusciferA Singularity - Re-Imagined by Carina Round'},
 {'id': '0zb2kpEQMnqJPiLACKMiFM',
  'name': "Slip Slidin' Away",
  'duration_ms': 285053,
  'popularity': 58,
  'disc_number': 1,
  'track_number': 7,
  'explicit': False,
  'type': 'track',
  'album_id': '4kdOH3s9cRL9YykvHFpSlD',
  'artist_id': '2CvCyf1gEVhI0mX6aFXmVI',
  'artist_count': 1,
  'uniqueName': "Paul SimonSlip Slidin' Away"},
 {'id': '6qLgGLeawKyhMFJ848XtYD',
  'name': 'Dragonaut',
  'duration_ms': 343160,
  'popularity': 52,
  'disc_number': 1,
  'track_number': 1,
  'explicit': False,
  'type': 'track',
  'album_id': '6L7ENQZtfSKVAq1CIRZSxs',
  'artist_id': '4Mt6w4tDGiPgV5q6JWPlrI',
  'a

In [93]:
row

uniqueName    PusciferMomma Sed (Alive At Club Nokia) - Live
artistName                                          Puscifer
trackName             Momma Sed (Alive At Club Nokia) - Live
Name: 4, dtype: object

In [92]:
r.json()

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=track%3AMomma+Sed+%28Alive+At+Club+Nokia%29+-+Live+artist%3APuscifer&type=track&offset=0&limit=1',
  'items': [],
  'limit': 1,
  'next': None,
  'offset': 0,
  'previous': None,
  'total': 0}}

In [43]:
sp = spotify_terminal()
r = sp.call_api_search('Dinner Bell','They Might Be Giants')
track_item = r.json()['tracks']['items'][0]
track_parsed = sp.parse_track_result(track_item)

https://api.spotify.com/v1/search?q=track%3ADinner%20Bell%20artist%3AThey%20Might%20Be%20Giants&type=track&limit=1


In [44]:
track_parsed

{'id': '0z6nT8XjlV0nrx35LxDki0',
 'name': 'Dinner Bell',
 'duration_ms': 128893,
 'popularity': 34,
 'disc_number': 1,
 'track_number': 9,
 'explicit': False,
 'type': 'track',
 'album_id': '5HswYyErxPHfQtHhNlzhnR',
 'artist_id': '6zB02lwP6L6ZH32nggQiJT',
 'artist_count': 1}

In [39]:
type(track_item)

dict

In [30]:
r.json()['tracks']['items'][0]['name']

'Artists Only - 2005 Remaster'

In [13]:
r = sp.call_api(['6wsqVwoiVH2kde4k4KKAFU','2wGSgTmgSF3xjRrHkTc25R'],'audio-features')

In [15]:
r.json()

{'audio_features': [{'danceability': 0.927,
   'energy': 0.619,
   'key': 5,
   'loudness': -4.441,
   'mode': 0,
   'speechiness': 0.0539,
   'acousticness': 0.0186,
   'instrumentalness': 0,
   'liveness': 0.104,
   'valence': 0.817,
   'tempo': 117.995,
   'type': 'audio_features',
   'id': '6wsqVwoiVH2kde4k4KKAFU',
   'uri': 'spotify:track:6wsqVwoiVH2kde4k4KKAFU',
   'track_href': 'https://api.spotify.com/v1/tracks/6wsqVwoiVH2kde4k4KKAFU',
   'analysis_url': 'https://api.spotify.com/v1/audio-analysis/6wsqVwoiVH2kde4k4KKAFU',
   'duration_ms': 211582,
   'time_signature': 4},
  {'danceability': 0.802,
   'energy': 0.785,
   'key': 9,
   'loudness': -4.781,
   'mode': 1,
   'speechiness': 0.207,
   'acousticness': 0.14,
   'instrumentalness': 0,
   'liveness': 0.123,
   'valence': 0.435,
   'tempo': 99.998,
   'type': 'audio_features',
   'id': '2wGSgTmgSF3xjRrHkTc25R',
   'uri': 'spotify:track:2wGSgTmgSF3xjRrHkTc25R',
   'track_href': 'https://api.spotify.com/v1/tracks/2wGSgTmgSF3xj